This notebook performs a linear model regression on timelapse data to understand the temporal contribution to each morphology feature.

Note that I also separate the disctinction between Cellprofiler (CP) and scDINO features.

## The linear model will be constructed as follows:
### $Y_{feature} = \beta_t X_t + \beta_{cell count} X_{cell count} + \beta_{Stuarosporine \space dose} X_{Stuarosporine \space dose} + \beta_0$




### The model explained
This model fits the factors to every individual morphology feature. 
This ulitmatly allows us to understand and interpret the contribution of each factor to the morphology feature.

### We define and interpret each term as follows:
- $Y_{feature}$: the morphology feature we are trying to predict
    - This is a morphology feature extracted from either CellProfiler or scDINO.
- $\beta_t$: the coefficient for the time variable
    - This coefficient represents the contribution of time to the morphology feature.
- $X_t$: the time variable
    - This variable represents the time elapsed from the start of imaging.
    - Note that these are timepoints starting at 0 and ending at 12 with increments of 1.
- $\beta_{cell count}$: the coefficient for the cell count variable
    - This coefficient represents the contribution of cell count/well to the morphology feature.
- $X_{cell count}$: the cell count variable
    - This variable represents the number of cells in each well.
- $\beta_{Stuarosporine \space dose}$: the coefficient for the Stuarosporine dose variable
    - This coefficient represents the contribution of Stuarosporine dose to the morphology feature.
- $X_{Stuarosporine \space dose}$: the Stuarosporine dose variable
    - This variable represents the Stuarosporine dose in each well.
    - note that the number input here is on a continuous scale with the attached units of nM.
- $\beta_0$: the intercept

#### Hypothesis:
The null hypothesis is that all factors for every feature will have a beta coeffienct of 0. This would imply that the factors do not contribute to the morphology feature.
The alternative hypothesis is that at least one factor for individual features will have a beta coeffienct > 0. This would imply that the factors do contribute to the morphology feature.

In [1]:
import pathlib
import warnings

import joblib
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
import tqdm
from statsmodels.stats.multitest import multipletests

warnings.filterwarnings("ignore")

In [2]:
def fit_linear_model(
    X: np.ndarray,
    y: np.ndarray,
    feature: str,
    write: bool = False,
) -> statsmodels.regression.linear_model.RegressionResultsWrapper:
    """
    Fit a linear model to the data and save the model to a file.

    Parameters
    ----------
    X : np.ndarray
        The input data to fit on. Should be numeric.
    y : np.ndarray
        The target data to fit on. Should be numeric.
    feature : str
        The feature name that is used from y to fit the model.
    shuffle : bool
        Whether to shuffle the data before fitting the model.

    Returns
    -------
    model : statsmodels.regression.linear_model.RegressionResultsWrapper
        The fitted model.
    """
    # Ensure X and y are numeric
    X = X.apply(pd.to_numeric, errors="coerce")
    y = pd.to_numeric(y, errors="coerce")

    # Drop rows with missing values and match the indices in the y series
    X = X.dropna()
    y = y.loc[X.index]

    # Add a constant for the intercept
    X = sm.add_constant(X)

    # Fit the model
    model = sm.OLS(y, X).fit()
    if write:
        # write the model to a file joblib
        joblib_path = pathlib.Path(f"../linear_models/lm_{feature}.joblib").resolve()
        joblib_path.parent.mkdir(parents=True, exist_ok=True)
        joblib.dump(model, joblib_path)

    return model

In [3]:
agg_profile_file_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs_aggregated.parquet"
).resolve(strict=True)

all_features_beta_df_path = pathlib.Path(
    "../results/all_features_beta_df.parquet"
).resolve()
all_features_beta_df_path.parent.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(agg_profile_file_path)
df.head()

,Metadata_Well,Metadata_Time,Metadata_dose,Metadata_number_of_singlecells,Metadata_plate,Metadata_compound,Metadata_control,Cells_AreaShape_Compactness_CP,Cells_AreaShape_Eccentricity_CP,Cells_AreaShape_Extent_CP,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,C-02,0.0,0.0,178.0,1,Staurosporine,negative,-0.225866,0.362134,0.195628,...,0.123994,-0.169231,-0.000171,-0.000714,0.219109,0.060974,0.073445,-0.099431,-0.202662,0.069599
1,C-02,1.0,0.0,181.0,1,Staurosporine,negative,-0.101367,0.212559,0.160123,...,-0.110603,0.076735,0.057752,0.065120,-0.253076,0.088509,-0.046158,0.118233,-0.090497,-0.116096
2,C-02,10.0,0.0,168.0,1,Staurosporine,negative,0.018758,0.438709,0.017725,...,-0.077459,-0.076050,0.179852,0.079523,-0.534580,0.115206,0.033339,0.179140,-0.031637,-0.142134
3,C-02,11.0,0.0,174.0,1,Staurosporine,negative,-0.041979,0.460182,-0.009282,...,-0.121237,-0.075781,0.197840,0.229394,-0.400539,0.074929,-0.064636,0.225648,-0.063553,-0.139911
4,C-02,12.0,0.0,175.0,1,Staurosporine,negative,0.045544,0.423336,0.057030,...,0.035974,-0.044032,0.490040,0.287544,-0.584386,0.145022,0.071476,0.225753,-0.012832,-0.152288


In [4]:
# get the metadata features
metadata_columns = [x for x in df.columns if "Metadata" in x]
# get the features
feature_columns = [x for x in df.columns if "Metadata" not in x]
time_column = "Metadata_Time"
single_cells_count_column = "Metadata_number_of_singlecells"
dose_column = "Metadata_dose"

# ensure that the interaction terms are both numeric
df["Metadata_number_of_singlecells"] = pd.to_numeric(
    df["Metadata_number_of_singlecells"], errors="coerce"
)
df["Metadata_Time"] = pd.to_numeric(df["Metadata_Time"], errors="coerce")
df["Metadata_dose"] = pd.to_numeric(df["Metadata_dose"], errors="coerce")

In [5]:
coefficient_names = {
    "beta": [],
    "p_value": [],
    "variate": [],
    "r2": [],
    "feature": [],
}

In [6]:
model_dict = {}

In [7]:
X = df[
    [
        time_column,
        single_cells_count_column,
        dose_column,
    ]
]
for feature in tqdm.tqdm(feature_columns):
    y = df[feature]
    model = fit_linear_model(
        X,
        y,
        feature,
    )
    # get the model coefficients and p-values
    for variate in model.params.keys():
        coefficient_names["beta"].append(model.params[variate])
        coefficient_names["variate"].append(variate)
    for pval in model.pvalues.keys():
        coefficient_names["p_value"].append(model.pvalues[pval])
        coefficient_names["r2"].append(model.rsquared)
        coefficient_names["feature"].append(feature)
    model_dict[feature] = model

100%|██████████| 2336/2336 [00:07<00:00, 299.85it/s]


In [8]:
# write the model to a file joblib
joblib_path = pathlib.Path("../linear_models/lm_all_features.joblib").resolve()
# write the model to a file
joblib_path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(model_dict, joblib_path)

['/home/lippincm/4TB_A/live_cell_timelapse_apoptosis_analysis/6.linear_modeling/linear_models/lm_all_features.joblib']

In [9]:
all_features_beta_df = pd.DataFrame.from_dict(coefficient_names)
# remove any "Metadata_" string from the feature names
all_features_beta_df["variate"] = all_features_beta_df["variate"].str.replace(
    "Metadata_", ""
)
all_features_beta_df["variate"] = all_features_beta_df["variate"].str.replace(
    "number_of_singlecells", "Cell count"
)
all_features_beta_df.head()

,beta,p_value,variate,r2,feature
0,0.115118,5.094561e-01,const,0.153274,Cells_AreaShape_Compactness_CP
1,-0.001031,7.989212e-01,Time,0.153274,Cells_AreaShape_Compactness_CP
2,0.000928,3.305227e-01,Cell count,0.153274,Cells_AreaShape_Compactness_CP
3,-0.002492,1.975790e-14,dose,0.153274,Cells_AreaShape_Compactness_CP
4,0.595095,2.961811e-19,const,0.067238,Cells_AreaShape_Eccentricity_CP


Extract feature information and save the dataframe

In [10]:
# split the df into two dfs, one with CP and with scDINO
cp_df = all_features_beta_df[all_features_beta_df["feature"].str.contains("CP")]

cp_df = cp_df.copy()
cp_df[
    [
        "Compartment",
        "Feature_type",
        "Measurement",
        "Channel",
        "extra1",
        "extra2",
        "extra3",
        "extra4",
        "extra5",
        "extra6",
    ]
] = cp_df["feature"].str.split("_", expand=True)
cp_df["featurizer_id"] = "CP"


# realign the channel and feature types - Cellprofiler outputs an unaligned df of feature names

# make the areashape channel None
cp_df.loc[cp_df["Feature_type"].str.contains("AreaShape"), "Channel"] = "None"

cp_df.loc[cp_df["Channel"].str.contains("CL", na=False), "Channel"] = (
    cp_df["Channel"].fillna("") + "_" + cp_df["extra1"].fillna("")
)
# merge the 488 with which emmission specrtra 1 or 2
cp_df.loc[cp_df["Channel"].str.contains("488", na=False), "Channel"] = (
    cp_df["Channel"].fillna("") + "_" + cp_df["extra2"].fillna("")
)

cp_df.loc[cp_df["extra1"].str.contains("CL", na=False), "extra1"] = (
    cp_df["extra1"].fillna("") + "_" + cp_df["extra2"].fillna("")
)
cp_df.loc[cp_df["extra1"].str.contains("488", na=False), "extra1"] = (
    cp_df["extra1"].fillna("") + "_" + cp_df["extra3"].fillna("")
)

# specify that it is ChomaLive (CL)
cp_df.loc[cp_df["extra2"].str.contains("CL", na=False), "extra2"] = (
    cp_df["extra2"].fillna("") + "_" + cp_df["extra3"].fillna("")
)
cp_df.loc[cp_df["extra2"].str.contains("488", na=False), "extra2"] = (
    cp_df["extra2"].fillna("") + "_" + cp_df["extra4"].fillna("")
)

cp_df.loc[cp_df["extra3"].str.contains("CL", na=False), "extra3"] = (
    cp_df["extra3"].fillna("") + "_" + cp_df["extra4"].fillna("")
)
cp_df.loc[cp_df["extra3"].str.contains("488", na=False), "extra3"] = (
    cp_df["extra3"].fillna("") + "_" + cp_df["extra5"].fillna("")
)

cp_df.loc[cp_df["extra4"].str.contains("CL", na=False), "extra4"] = (
    cp_df["extra4"].fillna("") + "_" + cp_df["extra5"].fillna("")
)
cp_df.loc[cp_df["extra4"].str.contains("488", na=False), "extra4"] = (
    cp_df["extra4"].fillna("") + "_" + cp_df["extra6"].fillna("")
)

cp_df.rename(columns={"extra1": "Channel2"}, inplace=True)
# remove the extra columns to retain the feature types and channels
cp_df.drop(
    columns=["Measurement", "extra2", "extra3", "extra4", "extra5", "extra6"],
    inplace=True,
)
# make channel2 None if feature is not correlation
cp_df.loc[~cp_df["Feature_type"].str.contains("Correlation"), "Channel2"] = "None"

# replace X or Y, or Adjacent with None in the Channel column
cp_df.loc[cp_df["Channel"].isin(["X", "Y", "Adjacent"]), "Channel"] = "None"

In [11]:
scdino_df = all_features_beta_df[all_features_beta_df["feature"].str.contains("scDINO")]
scdino_df = scdino_df.copy()
scdino_df["feature"] = scdino_df["feature"].str.replace("channel_", "")
scdino_df["feature"] = scdino_df["feature"].str.replace("channel", "")
scdino_df[["Channel", "remove", "feature", "feature_number", "featurizer_id"]] = (
    scdino_df["feature"].str.split("_", expand=True)
)
scdino_df.drop(columns=["remove"], inplace=True)
# set scDINO to be the feature type, Compartment and measurement
# as scDINO does not have these features only a channel and feature number
scdino_df[["Compartment", "Feature_type", "Measurement"]] = "scDINO"

In [12]:
final_df = pd.concat([cp_df, scdino_df], axis=0)
final_df["Channel"] = final_df["Channel"].str.replace("Adjacent", "None")
final_df["Channel"] = final_df["Channel"].str.replace("Y", "None")
final_df["Channel"] = final_df["Channel"].str.replace("CL_488_1", "488-1")
final_df["Channel"] = final_df["Channel"].str.replace("CL_488_2", "488-2")
final_df["Channel"] = final_df["Channel"].str.replace("CL_561", "561")
final_df["Channel"] = final_df["Channel"].str.replace("488-1", "CL 488-1")
final_df["Channel"] = final_df["Channel"].str.replace("488-2", "CL 488-2")
final_df["Channel"] = final_df["Channel"].str.replace("561", "CL 561")

In [13]:
final_df["variate"] = final_df["variate"].str.replace("dose", "Dose")

In [14]:
# correct the p-values using the Benjamini/Hochberg method
final_df["p_value_corrected"] = multipletests(final_df["p_value"], method="fdr_bh")[1]

In [15]:
# save the final df to a file
final_df.to_parquet(all_features_beta_df_path, index=False)
final_df.head()

,beta,p_value,variate,r2,feature,Compartment,Feature_type,Channel,Channel2,featurizer_id,feature_number,Measurement,p_value_corrected
0,0.115118,5.094561e-01,const,0.153274,Cells_AreaShape_Compactness_CP,Cells,AreaShape,None,None,CP,NaN,NaN,5.871186e-01
1,-0.001031,7.989212e-01,Time,0.153274,Cells_AreaShape_Compactness_CP,Cells,AreaShape,None,None,CP,NaN,NaN,8.423354e-01
2,0.000928,3.305227e-01,Cell count,0.153274,Cells_AreaShape_Compactness_CP,Cells,AreaShape,None,None,CP,NaN,NaN,4.094398e-01
3,-0.002492,1.975790e-14,Dose,0.153274,Cells_AreaShape_Compactness_CP,Cells,AreaShape,None,None,CP,NaN,NaN,6.772480e-14
4,0.595095,2.961811e-19,const,0.067238,Cells_AreaShape_Eccentricity_CP,Cells,AreaShape,None,None,CP,NaN,NaN,1.165969e-18
